In [7]:
import pandas as pd

# Replace 'path_to_your_dataset.csv' with the actual path to your dataset file
dataset_path = '/content/sentences.csv'

# Read the dataset into a pandas DataFrame
# You may need to adjust parameters like 'sep' depending on the format of your file
df = pd.read_csv(dataset_path, sep=',')
df.columns = ['english', 'darija']

# Example: Printing the first few entries
print(df.head())

# Now, extract the English and Darija sentences into lists
english_sentences = df['english'].tolist()
darija_sentences = df['darija'].tolist()

                                             english  \
0                They're hiding something, I'm sure!   
1    It's obvious they're trying to keep their cool.   
2            the hotels don't seem very comfortable.   
3  he is probably about to be laid off by head of...   
4                         of course he's depressive!   

                                   darija  
0    homa mkhbbyin chi haja, ana mti99en!  
1      bayna homa tay7awlo ib9aw mbrrdin.  
2  loTilat mabaynach fihom mori7in bzzaf.  
3      ghaliban ghayjrriw 3lih mn lkhdma!  
4                     Tab3an rah mkta2eb!  


In [8]:
df.head()


,english,darija
0,"They're hiding something, I'm sure!","homa mkhbbyin chi haja, ana mti99en!"
1,It's obvious they're trying to keep their cool.,bayna homa tay7awlo ib9aw mbrrdin.
2,the hotels don't seem very comfortable.,loTilat mabaynach fihom mori7in bzzaf.
3,he is probably about to be laid off by head of...,ghaliban ghayjrriw 3lih mn lkhdma!
4,of course he's depressive!,Tab3an rah mkta2eb!


In [9]:
target_texts=english_sentences
input_texts=darija_sentences

In [12]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np


# Parameters
batch_size = 64
epochs = 10
latent_dim = 256
num_samples = 10000

# Split data into training and testing sets
input_texts_train, input_texts_test, target_texts_train, target_texts_test = train_test_split(input_texts, target_texts, test_size=0.95)

# Fit a tokenizer on your data. Notice that the tokenizer is only fit on the training set
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts_train)
input_sequences_train = input_tokenizer.texts_to_sequences(input_texts_train)
input_sequences_test = input_tokenizer.texts_to_sequences(input_texts_test)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts_train)
target_sequences_train = target_tokenizer.texts_to_sequences(target_texts_train)
target_sequences_test = target_tokenizer.texts_to_sequences(target_texts_test)

num_encoder_tokens = len(input_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

# Pad sequences to the same length
max_sequence_length = max([len(seq) for seq in input_sequences_train])
encoder_input_data_train = pad_sequences(input_sequences_train, maxlen=max_sequence_length)
encoder_input_data_test = pad_sequences(input_sequences_test, maxlen=max_sequence_length)
decoder_input_data_train = pad_sequences(target_sequences_train, maxlen=max_sequence_length)
decoder_input_data_test = pad_sequences(target_sequences_test, maxlen=max_sequence_length)

# One-hot encode
encoder_input_data_train = to_categorical(encoder_input_data_train)
encoder_input_data_test = to_categorical(encoder_input_data_test)
decoder_input_data_train = to_categorical(decoder_input_data_train)
decoder_input_data_test = to_categorical(decoder_input_data_test)


# The architecture is the same
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Decoder_target_data_train will be ahead by one timestep and will not include the start character.
decoder_target_data_train = np.zeros(decoder_input_data_train.shape)
decoder_target_data_train[:,:-1,:] = decoder_input_data_train[:,1:,:]




In [13]:
model.fit([encoder_input_data_train, decoder_input_data_train], decoder_target_data_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([encoder_input_data_test, decoder_input_data_test], decoder_input_data_test))


Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 5.2127 - accuracy: 0.5674

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1917, in test_step
        y_pred = self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Inpu

In [ ]:
model.save('s2s.h5')

In [1]:
from transformers import AutoTokenizer, AutoModel
DarijaBERT_tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT")
DarijaBert_model = AutoModel.from_pretrained("SI2M-Lab/DarijaBERT")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


vocab.txt:   0%|          | 0.00/879k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


pytorch_model.bin:   0%|          | 0.00/836M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
